
# Export a session as Spike2 data with Photometry data


# Imports

In [2]:

# allow for automatic reloading of classes and function when updating the code
%load_ext autoreload
%autoreload 2 

# Import Session and Experiment class with helper functions
from trialexp.process.data_import import *
import pandas as pd
import datetime
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Variables

In [3]:

trial_window = [-2000, 6000]  # in ms

# time limit around trigger to perform an event
# determine successful trials
timelim = [0, 2000]  # in ms

# Digital channel nb of the pyphotometry device
# on which rsync signal is sent (from pycontrol device)
rsync_chan = 2

basefolder, _ = os.path.split(os.path.split(os.getcwd())[0])

# These must be absolute paths
# use this to use within package tasks files (in params)
tasksfile = os.path.join(basefolder, 'params\\tasks_params.csv')
# use this to put a local full path
#tasksfile = -r'C:/.../tasks_params.csv'

# photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\test_folder\photometry'
photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\pyphotometry\data\reaching_go_spout_incr_break2_nov22'
video_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\videos'


In [4]:
tasks = pd.read_csv(tasksfile, usecols=[1, 2, 3, 4], index_col=False)
tasks


,task,triggers,events,conditions
0,lick_go_nogo,go; nogo,bar; bar_off; spout,free_reward
1,lick_go_nogo_unconditionned,go; nogo,bar; bar_off; spout,free_reward
2,lick_go_nogo_unconditionned_opto,go; nogo,bar; bar_off; spout,free_reward
3,reaching_go_nogo,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
4,reaching_go_nogo_jc,CS_plus; CS_minus,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
5,reaching_go_nogo_opto_continuous,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset; s_go_sham; s_go_...
6,reaching_go_nogo_opto_sinusoid,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
7,reaching_go_nogo_opto_sinusoid_spout,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
8,reaching_go_nogo_reversal,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
9,reaching_go_nogo_reversal_incentive,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...


### Create an experiment object


In [5]:
# Folder of a full experimental batch, all animals included

# Enter absolute path like this
# pycontrol_files_path = r'T:\Data\head-fixed\test_folder\pycontrol'

# or this if you want to use data from the sample_data folder within the package
#pycontrol_files_path = os.path.join(basefolder, 'sample_data/pycontrol')
pycontrol_files_path = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22'

# Load all raw text sessions in the indicated folder or a sessions.pkl file
# if already existing in folder_path
exp_cohort = Experiment(pycontrol_files_path, update=True)  # TODO

# Only use if the Experiment cohort as been processed by trials before
# TODO: assess whether this can be removed or not
exp_cohort.by_trial = True


smrx_folder_path = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22\processed'


Saved sessions loaded from: sessions.pkl


In [6]:
exp_cohort.match_sessions_to_files(photometry_dir, ext='ppd')
exp_cohort.sync_photometry_files(2)
exp_cohort.save()


d:\onedrive - nexus365\private_dropbox\projects\trialexp\trialexp\utils\rsync.py:78: RuntimeWarning: invalid value encountered in log
  log_mse = np.log(chunk_mse)


saved \\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22\sessions.pkl


In [7]:

update_all_smrx = False

ss = exp_cohort.sessions

ss_ = [this_ss for this_ss in ss
       if (this_ss.subject_ID in [58, 60, 61, 62, 63, 64])
       and (this_ss.task_name == 'reaching_go_spout_incr_break2_nov22')
       and (this_ss.datetime.date() >= datetime.date(2023, 1, 26))]
ss_


In [8]:
exp_cohort.sessions = ss_


In [9]:
# Many combinations possible
conditions_dict0 = {'trigger': 'hold_for_water', 'valid': True}


# Aggregate all condition dictionaries in a list
condition_list = [conditions_dict0]
# Aliases for conditions
cond_aliases = [
    'any_trial',
]

# Groups as a list of lists
groups = None

# right_handed = [281]
# groups = [[280, 282, 299, 300, 301],\
#     [284, 285, 296, 297, 306, 307]]
# Window to exctract (in ms)


In [10]:
exp_cohort.sessions[0].print_lines[0:30]


['4003 T:2001 this trial> Success:X, 0 spouts\n#1: success:0, free:0, max allowed 6.0, next break 1.49 s, ---------X',
 '7490 T:5488 this trial> Success:X, 0 spouts\n#2: success:0, free:0, max allowed 6.0, next break 1.32 s, --------XX',
 '8954 water_on',
 '9632 water_on',
 '10811 T:8809 this trial> Success:O, 2 spouts\n#3: success:1, free:0, max allowed 6.0, next break 1.07 s, -------XXO',
 '13880 T:11878 this trial> Success:X, 0 spouts\n#4: success:1, free:0, max allowed 6.0, next break 1.21 s, ------XXOX',
 '17094 T:15092 this trial> Success:X, 0 spouts\n#5: success:1, free:0, max allowed 6.0, next break 1.15 s, -----XXOXX',
 '20246 T:18244 this trial> Success:X, 0 spouts\n#6: success:1, free:0, max allowed 6.0, next break 1.49 s, ----XXOXXX',
 '23739 T:21737 this trial> Success:X, 0 spouts\n#7: success:1, free:0, max allowed 6.0, next break 1.48 s, ---XXOXXXX',
 '27218 T:25216 this trial> Success:X, 0 spouts\n#8: success:1, free:0, max allowed 6.0, next break 1.43 s, --XXOXXXXX',
 

In [11]:
for ss in exp_cohort.sessions:
    smrxname = re.sub('\.txt', f'_{ss.task_name}.smrx', ss.file_name)
    print(smrxname)


kms058-2023-01-26-100537_reaching_go_spout_incr_break2_nov22.smrx
kms058-2023-01-26-100832_reaching_go_spout_incr_break2_nov22.smrx
kms064-2023-01-30-115942_reaching_go_spout_incr_break2_nov22.smrx
kms064-2023-01-31-103119_reaching_go_spout_incr_break2_nov22.smrx
kms064-2023-01-31-104217_reaching_go_spout_incr_break2_nov22.smrx
kms064-2023-02-01-105834_reaching_go_spout_incr_break2_nov22.smrx
kms063-2023-02-01-114040_reaching_go_spout_incr_break2_nov22.smrx
kms058-2023-02-01-115246_reaching_go_spout_incr_break2_nov22.smrx
kms058-2023-02-03-160440_reaching_go_spout_incr_break2_nov22.smrx
kms058-2023-02-03-162554_reaching_go_spout_incr_break2_nov22.smrx
kms063-2023-02-03-181228_reaching_go_spout_incr_break2_nov22.smrx
kms061-2023-02-03-181629_reaching_go_spout_incr_break2_nov22.smrx
kms061-2023-02-03-181725_reaching_go_spout_incr_break2_nov22.smrx
kms061-2023-02-06-103133_reaching_go_spout_incr_break2_nov22.smrx
kms058-2023-02-06-104132_reaching_go_spout_incr_break2_nov22.smrx
kms063-202

In [12]:
exp_cohort.sessions[0].print_lines[0]

a = re.sub('\n', '', exp_cohort.sessions[0].print_lines[0])

print(a)


4003 T:2001 this trial> Success:X, 0 spouts#1: success:0, free:0, max allowed 6.0, next break 1.49 s, ---------X


In [23]:
vars(exp_cohort.sessions[10])

{'file_name': 'kms063-2023-02-03-181228.txt',
 'experiment_name': 'reaching_go_spout_incr_break2_nov22',
 'task_name': 'reaching_go_spout_incr_break2_nov22',
 'setup_ID': 'COM8',
 'subject_ID': 63,
 'datetime': datetime.datetime(2023, 2, 3, 18, 12, 28),
 'datetime_string': '2023-02-03 18:12:28',
 'events': [Event(time=0, name='rsync'),
  Event(time=0, name='short_break'),
  Event(time=2000, name='CS_offset_timer'),
  Event(time=2001, name='busy_win'),
  Event(time=4001, name='busy_win_timer'),
  Event(time=4004, name='short_break'),
  Event(time=8493, name='busy_win'),
  Event(time=9346, name='rsync'),
  Event(time=10493, name='busy_win_timer'),
  Event(time=10496, name='short_break'),
  Event(time=10789, name='rsync'),
  Event(time=15530, name='busy_win'),
  Event(time=15872, name='button_press'),
  Event(time=15893, name='US_end_timer'),
  Event(time=15984, name='button_release'),
  Event(time=16046, name='button_press'),
  Event(time=16066, name='US_end_timer'),
  Event(time=16100, 

In [19]:

vars(exp_cohort.sessions[10].photometry_rsync)


{'pulse_times_A': array([      0,    9346,   10789,   16608,   19234,   28381,   34479,
          38782,   44871,   51778,   54015,   56288,   65130,   66820,
          68746,   70059,   74737,   82508,   85173,   86131,   93353,
          97728,  106168,  114230,  115061,  118555,  119532,  125624,
         132700,  138428,  144203,  145324,  153670,  163240,  170083,
         171029,  178096,  185134,  189366,  197053,  202554,  205073,
         207359,  211316,  220598,  229590,  231265,  235495,  238736,
         240426,  242112,  243299,  245414,  249132,  254319,  257795,
         265895,  275235,  282317,  282999,  290673,  298054,  299592,
         303957,  312104,  319385,  327386,  336140,  339597,  343159,
         343945,  351409,  353312,  356149,  362023,  365961,  367942,
         375714,  380245,  385281,  393120,  396320,  405454,  411150,
         419599,  427618,  436715,  438468,  441415,  443110,  448049,
         450280,  454124,  455802,  464497,  472012,  479760

In [29]:
i = 10

photometry_aligner = Rsync_aligner(exp_cohort.sessions[i].photometry_rsync.pulse_times_A, 
    exp_cohort.sessions[i].photometry_rsync.pulse_times_B,
    chunk_size=5, plot=False, raise_exception=True)
photometry_dict = import_ppd(exp_cohort.sessions[i].files['ppd'][0])
photometry_times_pyc = photometry_aligner.B_to_A(photometry_dict['time'])


In [45]:
exp_cohort.sessions[i].events_to_process

['bar', 'bar_off', 'spout', 'US_end_timer']

In [44]:
df_meta_photo, col_names_numpy, photometry_array, photometry_dict = get_photometry_trials(
    exp_cohort.sessions[i],
    conditions_list=condition_list,
    cond_aliases = cond_aliases,
    trial_window = trial_window,
    trig_on_ev = 'hold_for_water',
    last_before = None,
    baseline_low_pass = 0.001, # var name changed from former high-pass,
    # was misleading on baseline computation
    # see https://github.com/juliencarponcy/trialexp/issues/8
    # first fix 
    low_pass = 45, 
    median_filt = 3,
    motion_corr = True, 
    df_over_f = True, 
    z_score = True,
    downsampling_factor = 10, 
    return_full_session = True,
    export_vars = ['analog_1_df_over_f', 'zscored_df_over_f'],
    # remove_artifacts = remove_artifacts,
    verbose = True)



Exception: trig_on_ev not in events_to_process

#TODO

- How the pyphotometry data are stored?
- How can I align them with behaviour data?

In [ ]:
keys = [
        'button_press', 'bar', 'bar_off', 'spout', 'US_delay_timer', 'CS_offset_timer']

state_def = [
    {'name': 'busy_win',    'onset': 'busy_win',    'offset': 'short_break'},
    {'name': 'short_break', 'onset': 'short_break', 'offset': 'busy_win'}]

summary_df = pd.DataFrame()

for ss in exp_cohort.sessions:

    file_name = os.path.split(ss.file_name)
    file_name_ = re.sub('\.txt',  f'_{ss.task_name}.smrx', file_name[1])
    smrxname = os.path.join(smrx_folder_path, file_name_)
    print(smrxname)


    bw = ss.times['busy_win']
    sp = ss.times['spout']

    x_spout = [this_bw for this_bw in bw for spouts in sp if (
        spouts < this_bw) and (this_bw - spouts < 100)]

    x_bar = [this_bw for this_bw in bw if not any(
        [(spouts < this_bw) and (this_bw - spouts < 100) for spouts in sp])]
        
    event_ms = [{
        'name': 'triggered by spout',
        'time_ms': x_spout
    },
    {
        'name': 'triggered by bar_off',
        'time_ms': x_bar
    }
    ]

    if update_all_smrx or not os.path.isfile(smrxname):

        try:
            ss.plot_session(
                keys, state_def, export_smrx=True, event_ms=event_ms, smrx_filename= smrxname) #TODO

            summary_df = pd.concat([summary_df, 
                pd.DataFrame({
                    'file':ss.file_name,
                    'task':ss.task_name,
                    'triggered_by_spout': len(x_spout),
                    'triggered_by_bar_off': len(x_bar),
                    'reaching_trials': len(bw),
                    'trials': len(ss.times['busy_win'])},
                    index=[0])
                    ],
                    ignore_index=True)
        except Exception as err:
            print(f"Unexpected {err=}, {type(err)=}, for {file_name_}")

